In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

import panel as pn
pn.extension()

In [2]:
MAX_SEQUENCE_LENGTH = 512
device = torch.device('cpu')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
LEN_TOKENIZER = len(tokenizer)

configuration = GPT2Config(vocab_size=LEN_TOKENIZER, n_positions=MAX_SEQUENCE_LENGTH).from_pretrained('gpt2', output_hidden_states=True)

prompt = "<BOS>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

In [3]:
poem_stanza_model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
poem_stanza_model.resize_token_embeddings(LEN_TOKENIZER)
poem_stanza_model.load_state_dict(torch.load('poem_stanza_model.pth', map_location=torch.device('cpu')))
poem_stanza_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [33]:
import time 
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

def generate_output():
#     t0 = time.time()
    generated_outputs = poem_stanza_model.generate(generated, 
                                                   do_sample=True, 
                                                   top_k=50, 
                                                   max_length=MAX_SEQUENCE_LENGTH,
                                                   top_p=0.95, 
                                                   num_return_sequences=1
                                                  )

    for i, output in enumerate(generated_outputs):
#         print(format_time(time.time() - t0))
        return tokenizer.decode(output, skip_special_tokens=True)

In [24]:
print(generate_output())

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0:00:35
Vine, and thy hand would be cast down by the tide of the river, And the stream of Heaven, that is within its bounds
From the head of thy soul hath been a curtain,
On which Heaven hath shut thee down
Upon the waters around her throne
Into her throne there is no throne but thee.
Whose throne is she that reigns over her throne—
 Who in her throne shall it be?
How many are she whose throne—
 Who, though they may be old, but still be young,
 Shall still be—the bride that holds her hand—
 Who in her maidenhood a golden eagle,
 She hath a sweetheart who hangs her flowers.


In [34]:
text_panel = pn.pane.Markdown(object='')
button = pn.widgets.Button(name='Generate Poe Poetry', button_type='primary')

def click_cb(event):
    text_panel.object = generate_output()
    
button.on_click(click_cb)

In [38]:
app = pn.Column(button, text_panel)
app

Column
    [0] Button(button_type='primary', clicks=7, name='Generate Poe Poetry')
    [1] Markdown(str)

In [39]:
title = pn.pane.Markdown("# **Poe Poem Generator**")
desc = pn.pane.HTML("<p><b>Click the button to generate a stanza of Poe poetry (it will take about 30-40 seconds). </b></p>")
final_app = pn.Column(title, desc, app)

In [40]:
final_app.show()

Launching server at http://localhost:63025
